## Welcome to this course "Getting started with Apache Spark"
## Video: Summarize data in PySpark

![PySpark](https://drive.google.com/uc?id=1oU2tHXn4Tb4NJ0GQLbFQanLUVWj-3M-G)

## Contents
- Operations (Transformations and Actions)
  - Commonly used actions:
    - show: To show the dataframe output
    - collect: Run action command to compute and get the result
    - count: Count the numbers of rows
  - Commonly used transformations & functions:
    - distinct: Get unique values in a column
    - withColumn: Create new columns
      - Use popular functions like lit, when, etc.
    - withColumnRenamed: Rename column names
    - filter: Filter data using rules
    - groupby: Group and aggregate data by a column

## Setting up the PySpark environment
- Check out this video for more details: https://www.youtube.com/watch?v=r5PbUuLUZiE
  - You can check out the link in the description below
- You can use the below cell to install all the required libraries and files

In [1]:
# Setting up the PySpark environment

# Install java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Apache Spark binary: This link can change based on the version. Update this link with the latest version before using
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz

# Unzip file
!tar -xf spark-3.0.2-bin-hadoop2.7.tgz

# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

# Add environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

# findspark will locate spark in the system
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

### Initialize SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Hands-on PySpark on Google Colab") \
        .getOrCreate()

In [3]:
spark

### Read data
Dataset (In-vehicle coupon recommendation): https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv

In [5]:
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv -P sample_data/

In [6]:
# We can set header='true' and inferSchema='true' to infer the schema while reading the data

filepath = "sample_data/in-vehicle-coupon-recommendation.csv"
spark_df = spark.read.format('csv').options(header='true', inferSchema='true').load(filepath)
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|destination    |passanger|weather|temperature|time|coupon               |expiration|gender|age|maritalStatus    |has_children|education               |occupation|income         |car |Bar  |CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|Y  |
+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+--------

## Transformations and Actions

In [7]:
from pyspark.sql import functions as F

## Actions

### show

In [8]:
columns_to_use = ["destination", "passanger", "weather", "temperature", "time", "coupon", "gender", "age", "has_children", "income", "Y"]
spark_df = spark_df.select(*columns_to_use)
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
+---------------+---------+-------+-----------+-

### count: count the number of rows in a dataframe

In [9]:
spark_df.count(), len(spark_df.columns)

(12684, 11)

### collect: Run action command to compute and get the result

In [10]:
spark_df.agg(F.min("temperature"))

DataFrame[min(temperature): int]

In [11]:
spark_df.agg(F.min("temperature")).show()

+----------------+
|min(temperature)|
+----------------+
|              30|
+----------------+



In [13]:
spark_df.agg(F.min("temperature")).collect()[0][0]

30

In [14]:
spark_df.collect()[:5]

[Row(destination='No Urgent Place', passanger='Alone', weather='Sunny', temperature=55, time='2PM', coupon='Restaurant(<20)', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=1),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='10AM', coupon='Coffee House', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=0),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='10AM', coupon='Carry out & Take away', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=1),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='2PM', coupon='Coffee House', gender='Female', age='21', has_children=1, income='$37500 - $49999', Y=0),
 Row(destination='No Urgent Place', passanger='Friend(s)', weather='Sunny', temperature=80, time='2PM', coupon='Coffee House', gender='Female', age='21', has_children=1, income='$375

## Transformations

In [16]:
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
+---------------+---------+-------+-----------+-

### distinct: Get unique values in a columns

In [17]:
spark_df.select("gender").distinct().show()

+------+
|gender|
+------+
|Female|
|  Male|
+------+



In [18]:
spark_df.select("passanger").distinct().show()

+---------+
|passanger|
+---------+
|  Partner|
|    Alone|
|Friend(s)|
|   Kid(s)|
+---------+



In [19]:
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |
+---------------+---------+-------+-----------+-

### withColumn: Create new columns & withColumnRenamed
Use popular functions like lit, when, etc.

In [21]:
updated_spark_df = spark_df.withColumn("constant_column", F.lit("full_data"))
updated_spark_df.show(3, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |constant_column|
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |full_data      |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data      |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |full_data      |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+
only showing top 3 rows



In [22]:
updated_spark_df = updated_spark_df.withColumn("gender_mapped", F.when(F.col("gender") == "Female", 0)\
                                                                  .when(F.col("gender") == "Male", 1)\
                                                                  .otherwise(2))
updated_spark_df.sample(0.2).show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------+------+---+------------+---------------+---+---------------+-------------+
|destination    |passanger|weather|temperature|time|coupon         |gender|age|has_children|income         |Y  |constant_column|gender_mapped|
+---------------+---------+-------+-----------+----+---------------+------+---+------------+---------------+---+---------------+-------------+
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House   |Female|21 |1           |$37500 - $49999|0  |full_data      |0            |
|No Urgent Place|Kid(s)   |Sunny  |80         |10AM|Restaurant(<20)|Female|21 |1           |$37500 - $49999|1  |full_data      |0            |
|No Urgent Place|Kid(s)   |Sunny  |80         |2PM |Restaurant(<20)|Female|21 |1           |$37500 - $49999|1  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House   |Male  |21 |0           |$62500 - $74999|0  |full_data      |1            |

In [23]:
updated_spark_df = updated_spark_df.withColumnRenamed("passanger", "passenger")
updated_spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+-------------+
|destination    |passenger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |constant_column|gender_mapped|
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------------+-------------+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |full_data      |0            |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 -

In [24]:
updated_spark_df = updated_spark_df.withColumnRenamed("constant_column", "const_col")
updated_spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------+-------------+
|destination    |passenger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |const_col|gender_mapped|
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+---------+-------------+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |full_data|0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data|0            |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |full_data|0            |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |$37500 - $49999|0  |full_data|0            |

### filter: Filter data

In [25]:
updated_spark_df.select("temperature").distinct().show()

+-----------+
|temperature|
+-----------+
|         55|
|         80|
|         30|
+-----------+



In [26]:
temp_80_df = updated_spark_df.filter(F.col("temperature") == 80)
temp_80_df.show(3)

+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|    destination|passenger|weather|temperature|time|              coupon|gender|age|has_children|         income|  Y|const_col|gender_mapped|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|        Coffee House|Female| 21|           1|$37500 - $49999|  0|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|Carry out & Take ...|Female| 21|           1|$37500 - $49999|  1|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80| 2PM|        Coffee House|Female| 21|           1|$37500 - $49999|  0|full_data|            0|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
only s

In [27]:
updated_spark_df.count(), temp_80_df.count()

(12684, 6528)

In [28]:
temp_45_90_df = updated_spark_df.filter((F.col("temperature") > 45) & (F.col("temperature") < 90))
temp_45_90_df.show(3)

+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|    destination|passenger|weather|temperature|time|              coupon|gender|age|has_children|         income|  Y|const_col|gender_mapped|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
|No Urgent Place|    Alone|  Sunny|         55| 2PM|     Restaurant(<20)|Female| 21|           1|$37500 - $49999|  1|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|        Coffee House|Female| 21|           1|$37500 - $49999|  0|full_data|            0|
|No Urgent Place|Friend(s)|  Sunny|         80|10AM|Carry out & Take ...|Female| 21|           1|$37500 - $49999|  1|full_data|            0|
+---------------+---------+-------+-----------+----+--------------------+------+---+------------+---------------+---+---------+-------------+
only s

In [29]:
updated_spark_df.count(), temp_45_90_df.count()

(12684, 10368)

### groupby and aggregate

In [31]:
spark_df.show(3, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|income         |Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |$37500 - $49999|1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |$37500 - $49999|0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |$37500 - $49999|1  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+---------------+---+
only showing top 3 rows



In [33]:
spark_df.groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female| 6511|
|  Male| 6173|
+------+-----+



In [34]:
spark_df.groupby("temperature").count().show()

+-----------+-----+
|temperature|count|
+-----------+-----+
|         55| 3840|
|         80| 6528|
|         30| 2316|
+-----------+-----+



In [35]:
spark_df.groupby("gender").agg(F.min("temperature"), F.max("temperature")).show()

+------+----------------+----------------+
|gender|min(temperature)|max(temperature)|
+------+----------------+----------------+
|Female|              30|              80|
|  Male|              30|              80|
+------+----------------+----------------+



In [38]:
spark_df.groupby("gender").agg(F.min("temperature").alias("min_temperature"), F.max("temperature").alias("max_temperature")).show()

+------+---------------+---------------+
|gender|min_temperature|max_temperature|
+------+---------------+---------------+
|Female|             30|             80|
|  Male|             30|             80|
+------+---------------+---------------+



## Summary
- We have seen popular and basic transformations and action commands

### Thank you :)
-  That's the end of the this video. If you like this video, please do like, share and subscribe to my channel.
- If you are on LinkedIn, please tag me and share your thoughts on this video and the series "Getting started with PySpark - Hands on". This will motivate me to make more videos.
<div>
<img src="https://drive.google.com/uc?id=1ttB2gJaw0cXuJfj6GBx5VaYf2ArjiRXM" width="200"/>
</div>